# Wstęp do uczenia maszynowego - laboratorium 9

### Praca z "rzeczywistymi" danymi - wypożyczanie rowerów

## Eksploracyjna analiza danych

Zanim przejdziemy do wyboru i uczenia modeli, należy zapoznać się z danymi - dokonać tzw. **eksploracyjnej analizy danych**.

Pozwala ona wychwycić ewentualne problemy z danymi, np.:
- brakujące wartości  
- problematyczne formaty danych, np. kody pocztowe czasem jako string, czasem jako liczba
- niespójne wartości, np. "USA" i "United States" oznaczające to samo
- nierównomierny udział pewnych wartości/klas w danych
- nietypowe jednostki
- ...

Mówi się, że 80-90% pracy Data Scientista polega na czyszczeniu i obróbce danych. To bardzo zależy od projektu / od danych, ale jest w tym sporo prawdy.

Będziemy dziś pracować ze zbiorem danych dot. wypożyczania rowerów w Waszyngtonie: https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset.

Typowo jest on używany w zadaniu regresji - do przewidywania zapotrzebowania na rowery. Dziś użyjemy go jednak w zadaniu klasyfikacji.

### Bike Sharing Dataset - oficjalny opis z dokumentacji

```
Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv
- instant: record index
- dteday : date
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
- weathersit :
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
sns.set()
from sklearn.datasets import fetch_openml

### Ćwiczenie
Wczytać dane dot. dziennego wypożyczania rowerów (użyć pandas) do zmiennej `df`.

In [ ]:
df = pd.read_csv(filepath_or_buffer='./Data/WDUM8/day.csv')

### Ćwiczenie
Wyświetl 5 pierwszych, a następnie 3 ostatnie wiersze ze zbioru danych (pandas: `head`, `tail`).  
Zapoznaj się z danymi. Jaki dzień tygodnia oznaczony jest jako `0`?

In [ ]:
df.head()

In [ ]:
df.tail(n=3)

Weekday **0** - Niedziela

### Ćwiczenie
Użyj funkcji `df[nazwa_kolumny].plot()` z argumentem `figsize=(12,6)`, aby narysować wykres dla kolumny opisującej prędkość wiatru.

In [ ]:
df['windspeed'].plot(figsize=(12, 6))
plt.show()

### Ćwiczenie
Aby na osi X wyświetlić daty, najpierw wywołaj `df = df.set_index("dteday")`.

Następnie narysuj wykresy (kolejno) dla prędkości wiatru, wilgotności i temperatury (najlepiej wywoływać w osobnej komórce niż `set_index`).

In [ ]:
df = df.set_index('dteday')

In [ ]:
df['windspeed'].plot(figsize=(12, 6))
plt.show()

df['hum'].plot(figsize=(12, 6))
plt.show()

df['temp'].plot(figsize=(12, 6))
plt.show()

### Ćwiczenie
Do ramek danych można aplikować filtry w postaci warunków - uzyskamy w ten sposób ten podzbiór wierszy, który spełnia podany warunek.  
Aby uzyskać podzbiór wierszy, dla których dzień tygodnia to niedziela, można wywołać (dwa równoważne sposoby):  
- df.query("weekday == 0")  
- df[df.weekday == 0]

Wykorzystaj tę wiedzę, aby wyświetlić dni, w których łącznie wypożyczono mniej niż 500 rowerów.  
Czy zauważasz coś podejrzanego dla jednego z dni? Czy to błąd w danych? Co mogło się stać tego dnia? (można sprawdzić w Google)

In [ ]:
df[df['cnt'] < 500]

### Ćwiczenie
Narysuj wykres dla dwóch kolumn: temperatury i łącznej liczby wypożyczeń, aby zobaczyć, czy jest zależność.

hint: `df[lista_kolumn]`

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

df['temp'].plot(ax=ax1, color='tab:orange', label='Temperature')

# Create a separate axis to plot the rental counts

ax2 = ax1.twinx()

df['cnt'].plot(ax=ax2, color='tab:blue', label='Count')

ax1.set_ylabel('Temparature', color='tab:orange')
ax2.set_ylabel('Count', color='tab:blue')

# Set legends

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()

### Ćwiczenie
Dodaj nową kolumnę `cnt_scaled` do ramki danych, wstawiając tam wartości z `cnt` podzielone przez maksymalną wartość kolumny `cnt`.

Następnie narysuj na jednym wykresie temperaturę i `cnt_scaled`.

Na koniec usuń kolumnę `cnt_scaled` wywołując `df.drop(["cnt_scaled"], axis=1, inplace=True)`

In [ ]:
df['cnt_scaled'] = df['cnt'] / df['cnt'].max()
df.head()

In [ ]:
col_list = ['temp', 'cnt_scaled']

df[col_list].plot(figsize=(12, 6))
plt.show()

In [ ]:
df.drop(labels=col_list[1], axis=1, inplace=True)
df.head()

### Ćwiczenie
Uruchom funkcję `df.descirbe()`, aby zobaczyć statystyki (min, max, średnia, mediana, kwartyle) dla każdej kolumny.

In [ ]:
df.describe()

### Ćwiczenie
Narysuj wykres typu scatterplot (XY) dla temperatury na osi X i wilgotności na osi Y.

`df.plot(kind='scatter', x=kolumna1, y=kolumna2)`

In [ ]:
df.plot(kind='scatter', x='temp', y='hum')
plt.show()

### Ćwiczenie
Zwizualizuj korelacje między poszczególnymi cechami z użyciem funkcji `sns.heatmap()` podając jako jej argument `df.corr()`.

In [ ]:
sns.heatmap(data=df.corr())

### Ćwiczenie
Narysuj wykres typu "catplot" (uruchom gotowy kod).

In [ ]:
sns.catplot(data=df,                                   # dataframe
            x="weekday", y="cnt",                      # parametry wymagane
            col="season",                              # parametry opcjonalne
            col_wrap=4,
            order=[0, 1, 2, 3, 4, 5, 6],               # domyślnie wartości "weekday" nie są posortowane
            kind="bar")                                # typ wykresu

### Ćwiczenie
Znajdź średnią liczbę dziennych wypożyczeń dla każdej z pór roku. Wykorzystaj `df.groupby()`

In [ ]:
df_grouped = df.groupby(by='season', axis=0)
avg_rental_counts_by_season = df_grouped['cnt'].mean()
avg_rental_counts_by_season

### Ćwiczenie
Uruchom poniższy kod, aby narysować rozkłady temperatur dla każdej pory roku.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
# temperatures in seasons
colors = {1: "#5555dd", 2: "#55dd55", 3: "#fcc969", 4: "#dd5555"}
for season, season_df in df.groupby("season")["temp"]:
    sns.distplot(season_df, hist=True, label=season, color=colors[season], ax=ax)

## Uczenie z nadzorem - klasyfikacja

Po zapoznaniu się z danymi dokonamy teraz klasyfikacji (binarnej), czy dany dzień jest dniem zimowym.


### Walidacja

Należy podzielić zbiór danych na uczący i testowy (prosta walidacja).  
Jak było podane na wykładzie, jednym z "najmniej złych" podziałów tego zbioru danych jest wzięcie roku 2011 jako zbioru uczącego i 2012 jako testowego.

In [ ]:
df = pd.read_csv('./Data/WDUM8/day.csv') # jeszcze raz wczytujemy dane, bo wcześniej usunęliśmy kolumnę `dteday`
train_mask = df.dteday < '2012-01-01'

df_train = df[train_mask]
df_test = df[~train_mask]

len(df_train), len(df_test)

### Ćwiczenie

Przygotuj dane uczące i testowe:
- zostaw jedynie dwie cechy: temperaturę i liczbę "przypadkowych" użytkowników (hint `new_df = df[lista_kolumn]`
- stwórz "etykiety" - czy dany wiersz jest dniem zimowym

Nazwij zmienne: `X_train`, `X_test`, `Y_train` oraz `Y_test`

In [ ]:
new_df = df[['temp', 'casual']]
new_df

In [ ]:
# Add a 'winter_day' label

normalized_temp_threshold = 0.276595745 # 5 degrees Celsius

new_df.loc[:, 'winter_day'] = (new_df['temp'] < normalized_temp_threshold).astype(int)
new_df

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data

X = new_df.drop(columns=['winter_day'])
y = new_df['winter_day']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
print(X_train.shape)
print(X_test.shape)

### Standaryzacja

### Ćwiczenie

Który (tylko jeden) z poniższych wariantów standaryzacji jest prawidłowy i dlaczego? Wybierz właściwy i uruchom w komórce poniżej.

1.
```
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()
```

2.
```
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()
```

3.
```
X_test = (X_test - X_train.mean()) / X_train.std()
X_train = (X_train - X_train.mean()) / X_train.std()
```

Według wzoru na standaryzację obliczamy daną wartość **X**, minus **wartość średnią X** i dzielimy przez **odchylenie X**

![Wzór na standardyzację](Data/WDUM8/formula.png)

In [ ]:
# Select the 1st variant

X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

### Algorytm k najbliższych sąsiadów

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=3)

### Ćwiczenie
Na przygotowanych danych wytrenuj model klasyfikujący dni na zimowe i niezimowe oparty na algorytmie k najbliższych sąsiadów dla k=3.

In [ ]:
KNN.fit(X=X_train, y=Y_train)

### Ćwiczenie
Sprawdź skuteczność (accuracy) modelu.

Uwaga - na poprzednich zajęciach robiliśmy tak:
```
from sklearn.metrics import accuracy_score
preds = model.predict(cechy_testowe)
print(accuracy_score(preds, etykiety_testowe)
```

Da się prościej:

```
print(model.score(cechy_testowe, etykiety_testowe))
```

In [ ]:
from sklearn.metrics import accuracy_score

preds = KNN.predict(X=X_test)
print(f'Model accuracy: {accuracy_score(y_pred=preds, y_true=Y_test)}')

In [ ]:
print(f'Model accuracy: {KNN.score(X=X_test, y=Y_test)}')

### Ćwiczenie

Uruchom poniższy kod, aby znaleźć wartość k, dla którego skuteczność modelu jest najlepsza.

In [ ]:
test_score_list = []
k_list = range(1, 201)

for k in k_list:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    test_score_list.append(knn.score(X_test, Y_test))

k_list[np.argmax(test_score_list)], test_score_list[np.argmax(test_score_list)]

### Ćwiczenie

Uruchom poniższą komórkę, aby narysować zależność skuteczności od k. Wyjaśnij, co dzieje się dla zbyt wysokich wartości k.

hint: `Y_train.value_counts()`

In [ ]:
sns.lineplot(x=k_list, y=test_score_list)

In [ ]:
Y_train.value_counts()

Dla zbyt wysokich wartości k-sąsiadów wyniki zostają przekłamane przez dominującą ilość 0 w zbiorze danych.

### Ćwiczenie

Jaką skuteczność miałby najlepszy możliwy model stały (tzn. taki, który zawsze zwraca tę samą wartość)?

Dla modeli z danymi o dwóch równych klasach byłoby to **50%**.

Dla modeli z danymi o nierównych klasach byłby to **procent najczęściej występującej klasy**.